In [49]:
%pylab inline
from smalllib import ff, of
import shutil

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['exp', 'inf', 'asarray', 'flag', 'zipf', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [50]:
import attr

@attr.attrs
class ArchivableFile:
    filename = attr.ib()
    extend_to_aux = attr.ib(default=False)
    group = attr.ib(default=None)
    rename_to= attr.ib(default=None)
    is_aux = attr.ib(default=False)
    
    

In [51]:
files = [ArchivableFile(of("crater_cropped_dem_textured.obj"), True , "model/meshes"),
        ArchivableFile(of("radargram.obj"), True , "model/meshes")]

In [52]:
for i in [0,1,2]:
    files.append(ArchivableFile(of(f"tube_{i}.obj"), False, "model/meshes", f"plane_topography_intersection_horizon_{i}.obj"))
    
for i in [0,1,2]:
    files.append(ArchivableFile(of(f"large_plane_{i}.obj"), False, "model/meshes", f"modelled_horizon_{i}.obj"))

In [53]:
vectors = [["prediction.gpkg", "predicted_intersections.gpkg"],
          ["horizons.gpkg","horizons_2.5D.gpkg"],
          ["path.gpkg","yutu_path_gereferenced.gpkg"]]

rasters = [["path_georef.tif", "xiao_georeferenced.tif"]]

In [54]:
for inf, out in vectors:
    files.append(ArchivableFile(of(inf), False, "vector", out))
    
for inf, out in rasters:
    files.append(ArchivableFile(of(inf), False, "raster", out))
    
    
files.append(ArchivableFile("../package_README.md", False, "", f"README.md"))

In [55]:
out_folder = "../PM-MOO-D-YutuGPR"
import os
os.makedirs(out_folder, exist_ok=True)


In [56]:
import glob

In [57]:
auxes = []
exclude_exts = [".vtp"]
for f in files:
#     print(f.filename)
    
    if f.extend_to_aux:
        basename = os.path.basename(f.filename)
        print(basename)
        path = os.path.dirname(f.filename)
        print(path)
        noextname = os.path.splitext(basename)[0]
        aux = glob.glob(f"{path}/{noextname}.*")
        
        aux.remove(f.filename)
        for au in aux:
            skip=False
            for ex in exclude_exts:
                if au.endswith(ex):
                    skip = True
                    
            if skip:
                continue
            from copy import deepcopy
            newf = deepcopy(f)
            newf.filename =au
            newf.extend_to_aux = False
            newf.is_aux = True
            auxes.append(newf)
            
        print(aux)

        
files += auxes

crater_cropped_dem_textured.obj
/home/luca/data/Code/yutu_radagram_3dmodelling.git/output
['/home/luca/data/Code/yutu_radagram_3dmodelling.git/output/crater_cropped_dem_textured.mtl', '/home/luca/data/Code/yutu_radagram_3dmodelling.git/output/crater_cropped_dem_textured.png']
radargram.obj
/home/luca/data/Code/yutu_radagram_3dmodelling.git/output
['/home/luca/data/Code/yutu_radagram_3dmodelling.git/output/radargram.png', '/home/luca/data/Code/yutu_radagram_3dmodelling.git/output/radargram.vtp', '/home/luca/data/Code/yutu_radagram_3dmodelling.git/output/radargram.mtl']


In [58]:
for f in files:
    flag = f"AUX: [{f.is_aux}]"
    print(f"Processing {f.filename} {flag}")
    
    if not os.path.exists(f.filename):
        print("File do not exist. aboorting")
        break
        
    outf = out_folder + f"/{f.group}/"
    print(f"oufolder will be {outf}" )
    if f.rename_to is not None:
        outfile = f.rename_to
    else:
        outfile = os.path.basename(f.filename)
        
    print(f"outfile {outfile}")
    os.makedirs(outf, exist_ok=True)
    fullout = os.path.join(outf, outfile)
    
    shutil.copy2(f.filename, fullout)
    print(f"expected output {fullout}\n")
        
    

Processing /home/luca/data/Code/yutu_radagram_3dmodelling.git/output/crater_cropped_dem_textured.obj AUX: [False]
oufolder will be ../PM-MOO-D-YutuGPR/model/meshes/
outfile crater_cropped_dem_textured.obj
expected output ../PM-MOO-D-YutuGPR/model/meshes/crater_cropped_dem_textured.obj

Processing /home/luca/data/Code/yutu_radagram_3dmodelling.git/output/radargram.obj AUX: [False]
oufolder will be ../PM-MOO-D-YutuGPR/model/meshes/
outfile radargram.obj
expected output ../PM-MOO-D-YutuGPR/model/meshes/radargram.obj

Processing /home/luca/data/Code/yutu_radagram_3dmodelling.git/output/tube_0.obj AUX: [False]
oufolder will be ../PM-MOO-D-YutuGPR/model/meshes/
outfile plane_topography_intersection_horizon_0.obj
expected output ../PM-MOO-D-YutuGPR/model/meshes/plane_topography_intersection_horizon_0.obj

Processing /home/luca/data/Code/yutu_radagram_3dmodelling.git/output/tube_1.obj AUX: [False]
oufolder will be ../PM-MOO-D-YutuGPR/model/meshes/
outfile plane_topography_intersection_horizon_

# some info to use in the package readme

In [59]:
bounds = "../images/crater_bounds.gpkg"
import geopandas as gpd
ext = gpd.read_file(bounds)

In [60]:
aslatlon = ext.to_crs("ESRI:104903")
print(aslatlon.bounds)
asarray = np.array(aslatlon.bounds)[0]
minlon, minlat, maxlon, maxlat = asarray

        minx       miny      maxx       maxy
0 -19.536877  44.113296 -19.50931  44.128802


In [61]:
if minlon < 0:
    minlon = 360 +minlon
    
if maxlon <0:
    maxlon = 360+maxlon

In [62]:
print(minlon, minlat, maxlon, maxlat)

340.4631232287061 44.11329617834599 340.4906900727858 44.128801823072436


In [63]:
def decdeg2dms(dd):
    mnt,sec = divmod(dd*3600,60)
    deg,mnt = divmod(mnt,60)
    return deg,mnt,sec

def format_degs(d,m,s):
    return f"{int(d)}d{int(m)}" + "'" +"{:.2f}\"".format(s)

In [64]:
exp = [format_degs(*decdeg2dms(val)) for val in asarray]
[print(ex) for ex in exp];

-20d27'47.24"
44d6'47.87"
-20d29'26.48"
44d7'43.69"


In [65]:
cs = ext.crs

In [66]:
cs.to_proj4()

<ipython-input-66-af9c76a80d2d>:1: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  cs.to_proj4()


'+proj=eqc +lat_ts=44 +lat_0=0 +lon_0=180 +x_0=0 +y_0=0 +R=1737400 +units=m +no_defs +type=crs'

# Compress everything

In [67]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        print(root)
        for file in files:
            fullpath = os.path.join(root, file)
            ziph.write(fullpath, arcname=fullpath[len(out_folder) :])

zipf = zipfile.ZipFile(f"{out_folder}.zip", 'w', zipfile.ZIP_DEFLATED)
zipdir(out_folder, zipf)
zipf.close()

../PM-MOO-D-YutuGPR
../PM-MOO-D-YutuGPR/model
../PM-MOO-D-YutuGPR/model/meshes
../PM-MOO-D-YutuGPR/raster
../PM-MOO-D-YutuGPR/vector
